#### ANALISIS DE WITH COLUM SIMPLE

In [2]:
df = sc.parallelize([
    ("orange", "apple"), ("kiwi", None), (None, "banana"), 
    ("mango", "mango"), (None, None)
]).toDF(["fruit1", "fruit2"])

df.withColumn('waaaaaaaaaa', df.fruit2).show()

+------+------+-----------+
fruit1|fruit2|waaaaaaaaaa|
+------+------+-----------+
orange| apple| apple|
 kiwi| null| null|
 null|banana| banana|
 mango| mango| mango|
 null| null| null|
+------+------+-----------+

### PRUBEAS PARAMETROS WORD2VECT

In [4]:
from pyspark.sql.functions import udf

# UDF PARA OTENER TAMANNO DE ELEMENTOS LISTA
@udf("int")
def len_list(_list):
  return len(_list)

In [5]:
sent = ("a b " * 100 + "a c " * 10).split(" ")
sent2 = ("a b " * 99 + "a c " * 11).split(" ")
sent3 = ("a b " * 101 + "a c " * 9).split(" ")

sent4 = ("e f " * 100 + "a k " * 10).split(" ")
sent5 = ("e f " * 99 + "a k " * 11).split(" ")
sent6 = ("e f " * 101 + "a k " * 9).split(" ")

doc = spark.createDataFrame(
  [(sent,), (sent2,), (sent3,), (sent4,), (sent5,), (sent6,)], 
  ["sentence"]
)
doc = doc.withColumn("len_sentence", len_list(doc['sentence']))
doc.show()

+--------------------+------------+
 sentence|len_sentence|
+--------------------+------------+
[a, b, a, b, a, b...| 221|
[a, b, a, b, a, b...| 221|
[a, b, a, b, a, b...| 221|
[e, f, e, f, e, f...| 221|
[e, f, e, f, e, f...| 221|
[e, f, e, f, e, f...| 221|
+--------------------+------------+

In [6]:
from pyspark.ml.feature import Word2Vec

help(Word2Vec)

Help on class Word2Vec in module pyspark.ml.feature:

class Word2Vec(pyspark.ml.wrapper.JavaEstimator, pyspark.ml.param.shared.HasStepSize, pyspark.ml.param.shared.HasMaxIter, pyspark.ml.param.shared.HasSeed, pyspark.ml.param.shared.HasInputCol, pyspark.ml.param.shared.HasOutputCol, pyspark.ml.util.JavaMLReadable, pyspark.ml.util.JavaMLWritable)
 Word2Vec trains a model of `Map(String, Vector)`, i.e. transforms a word into a code for further
 natural language processing or machine learning process.
 
 >>> sent = ("a b " * 100 + "a c " * 10).split(" ")
 >>> doc = spark.createDataFrame([(sent,), (sent,)], ["sentence"])
 >>> word2Vec = Word2Vec(vectorSize=5, seed=42, inputCol="sentence", outputCol="model")
 >>> model = word2Vec.fit(doc)
 >>> model.getVectors().show()
 +----+--------------------+
 |word| vector|
 +----+--------------------+
 | a|[0.09461779892444...|
 | b|[1.15474212169647...|
 | c|[-0.3794820010662...|
 +----+--------------------+
 ...
 >>> model.findSynonymsArray("a", 2)
 [('b', 0.25053444504737854), ('c', -0.6980510950088501)]
 >>> from pyspark.sql.functions import format_number as fmt
 >>> model.findSynonyms("a", 2).select("word", fmt("similarity", 5).alias("similarity")).show()
 +----+----------+
 |word|similarity|
 +----+----------+
 | b| 0.25053|
 | c| -0.69805|
 +----+----------+
 ...
 >>> model.transform(doc).head().model
 DenseVector([0.5524, -0.4995, -0.3599, 0.0241, 0.3461])
 >>> word2vecPath = temp_path + "/word2vec"
 >>> word2Vec.save(word2vecPath)
 >>> loadedWord2Vec = Word2Vec.load(word2vecPath)
 >>> loadedWord2Vec.getVectorSize() == word2Vec.getVectorSize()
 True
 >>> loadedWord2Vec.getNumPartitions() == word2Vec.getNumPartitions()
 True
 >>> loadedWord2Vec.getMinCount() == word2Vec.getMinCount()
 True
 >>> modelPath = temp_path + "/word2vec-model"
 >>> model.save(modelPath)
 >>> loadedModel = Word2VecModel.load(modelPath)
 >>> loadedModel.getVectors().first().word == model.getVectors().first().word
 True
 >>> loadedModel.getVectors().first().vector == model.getVectors().first().vector
 True
 
 .. versionadded:: 1.4.0
 
 Method resolution order:
 Word2Vec
 pyspark.ml.wrapper.JavaEstimator
 pyspark.ml.wrapper.JavaParams
 pyspark.ml.wrapper.JavaWrapper
 pyspark.ml.base.Estimator
 pyspark.ml.param.shared.HasStepSize
 pyspark.ml.param.shared.HasMaxIter
 pyspark.ml.param.shared.HasSeed
 pyspark.ml.param.shared.HasInputCol
 pyspark.ml.param.shared.HasOutputCol
 pyspark.ml.param.Params
 pyspark.ml.util.Identifiable
 pyspark.ml.util.JavaMLReadable
 pyspark.ml.util.MLReadable
 pyspark.ml.util.JavaMLWritable
 pyspark.ml.util.MLWritable
 builtins.object
 
 Methods defined here:
 
 __init__(self, vectorSize=100, minCount=5, numPartitions=1, stepSize=0.025, maxIter=1, seed=None, inputCol=None, outputCol=None, windowSize=5, maxSentenceLength=1000)
 __init__(self, vectorSize=100, minCount=5, numPartitions=1, stepSize=0.025, maxIter=1, seed=None, inputCol=None, outputCol=None, windowSize=5, maxSentenceLength=1000)
 
 getMaxSentenceLength(self)
 Gets the value of maxSentenceLength or its default value.
 
 .. versionadded:: 2.0.0
 
 getMinCount(self)
 Gets the value of minCount or its default value.
 
 .. versionadded:: 1.4.0
 
 getNumPartitions(self)
 Gets the value of numPartitions or its default value.
 
 .. versionadded:: 1.4.0
 
 getVectorSize(self)
 Gets the value of vectorSize or its default value.
 
 .. versionadded:: 1.4.0
 
 getWindowSize(self)
 Gets the value of windowSize or its default value.
 
 .. versionadded:: 2.0.0
 
 setMaxSentenceLength(self, value)
 Sets the value of :py:attr:`maxSentenceLength`.
 
 .. versionadded:: 2.0.0
 
 setMinCount(self, value)
 Sets the value of :py:attr:`minCount`.
 
 .. versionadded:: 1.4.0
 
 setNumPartitions(self, value)
 Sets the value of :py:attr:`numPartitions`.
 
 .. versionadded:: 1.4.0
 
 setParams(self, vectorSize=100, minCount=5, numPartitions=1, stepSize=0.025, maxIter=1, seed=None, inputCol=None, outputCol=None, windowSize=5, maxSentenceLength=1000)
 setPara

In [7]:
%%time
word2Vec = Word2Vec(vectorSize=100, seed=42, inputCol="sentence", outputCol="model",  maxIter=1)
model = word2Vec.fit(doc)

vv = model.getVectors()
vv = vv.withColumn("len_vector", len_list(vv['vector']))
vv.show()

+----+--------------------+----------+
word| vector|len_vector|
+----+--------------------+----------+
 a|[0.02185205556452...| 100|
 b|[0.15395373106002...| 100|
 c|[-0.0455951876938...| 100|
+----+--------------------+----------+

CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 1.04 s

In [8]:
%%time
word2Vec = Word2Vec(vectorSize=5, seed=42, inputCol="sentence", outputCol="model",  maxIter=1)
model = word2Vec.fit(doc)

vv = model.getVectors()
vv = vv.withColumn("len_vector", len_list(vv['vector']))
vv.show()

+----+--------------------+----------+
word| vector|len_vector|
+----+--------------------+----------+
 a|[0.06063261628150...| 5|
 b|[1.15109908580780...| 5|
 c|[-0.3636681437492...| 5|
+----+--------------------+----------+

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 646 ms

In [9]:
%%time
word2Vec = Word2Vec(vectorSize=100, seed=42, inputCol="sentence", outputCol="model",  maxIter=5)
model = word2Vec.fit(doc)

vv = model.getVectors()
vv = vv.withColumn("len_vector", len_list(vv['vector']))
vv.show()

+----+--------------------+----------+
word| vector|len_vector|
+----+--------------------+----------+
 a|[-0.0391055643558...| 100|
 b|[0.15813410282135...| 100|
 c|[-0.0872000753879...| 100|
+----+--------------------+----------+

CPU times: user 16 ms, sys: 4 ms, total: 20 ms
Wall time: 1.42 s

In [10]:
%%time
word2Vec = Word2Vec(vectorSize=100, seed=42, inputCol="sentence", outputCol="model",  maxIter=10)
model = word2Vec.fit(doc)

vv = model.getVectors()
vv = vv.withColumn("len_vector", len_list(vv['vector']))
vv.show()

+----+--------------------+----------+
word| vector|len_vector|
+----+--------------------+----------+
 a|[-0.0573107227683...| 100|
 b|[0.13437749445438...| 100|
 c|[-0.0847115665674...| 100|
+----+--------------------+----------+

CPU times: user 12 ms, sys: 8 ms, total: 20 ms
Wall time: 1.35 s

In [11]:
%%time
word2Vec = Word2Vec(vectorSize=100, seed=42, inputCol="sentence", outputCol="model",  maxIter=1, numPartitions=1)
model = word2Vec.fit(doc)

vv = model.getVectors()
vv = vv.withColumn("len_vector", len_list(vv['vector']))
vv.show()

+----+--------------------+----------+
word| vector|len_vector|
+----+--------------------+----------+
 a|[0.02185205556452...| 100|
 b|[0.15395373106002...| 100|
 c|[-0.0455951876938...| 100|
+----+--------------------+----------+

CPU times: user 20 ms, sys: 8 ms, total: 28 ms
Wall time: 1 s

In [12]:
%%time
word2Vec = Word2Vec(vectorSize=100, seed=42, inputCol="sentence", outputCol="model",  maxIter=1, numPartitions=2)
model = word2Vec.fit(doc)

vv = model.getVectors()
vv = vv.withColumn("len_vector", len_list(vv['vector']))
vv.show()

+----+--------------------+----------+
word| vector|len_vector|
+----+--------------------+----------+
 a|[0.01315427385270...| 100|
 b|[0.30133217573165...| 100|
 c|[-0.0930862948298...| 100|
+----+--------------------+----------+

CPU times: user 16 ms, sys: 8 ms, total: 24 ms
Wall time: 820 ms